In [33]:
from scipy.sparse import csr_matrix,rand,coo_matrix
import tables as tb
import numpy as np
import pandas as pd
from numpy import array

In [2]:
file=pd.read_table("/Users/jma7/Development/hail/test/genotype.tsv",index_col=0)

In [24]:
sparsefile=file.to_sparse(fill_value=0)

In [34]:
def sparse_df_to_array(df):
    num_rows = df.shape[0]   

    data = []
    row = []
    col = []

    for i, col_name in enumerate(df.columns):
        if isinstance(df[col_name], pd.SparseSeries):
            column_index = df[col_name].sp_index
            if isinstance(column_index, pd._sparse.BlockIndex):
                column_index = column_index.to_int_index()

            ix = column_index.indices
            data.append(df[col_name].sp_values)
            row.append(ix)
            col.append(len(df[col_name].sp_values) * [i])
        else:
            data.append(df[col_name].values)
            row.append(np.array(range(0, num_rows)))
            col.append(np.array(num_rows * [i]))

    data_f = np.concatenate(data)
    row_f = np.concatenate(row)
    col_f = np.concatenate(col)

    arr = coo_matrix((data_f, (row_f, col_f)), df.shape, dtype=np.float64)
    return arr.tocsr()

In [4]:
mat=csr_matrix(file.values)

In [46]:
def store_sparse_mat(m, name, store='store.h5'):
    msg = "This code only works for csr matrices"
    assert(m.__class__ == csr_matrix), msg
    with tb.open_file(store,'a') as f:
        for par in ('data', 'indices', 'indptr', 'shape'):
            full_name = '%s_%s' % (name, par)
            try:
                n = getattr(f.root, full_name)
                n._f_remove()
            except AttributeError:
                pass

            arr = array(getattr(m, par))
            atom = tb.Atom.from_dtype(arr.dtype)
            ds = f.create_carray(f.root, full_name, atom, arr.shape)
            print(f)
            ds[:] = arr

def load_sparse_mat(name, store='store.h5'):
    with tb.open_file(store) as f:
        pars = []
        for par in ('data', 'indices', 'indptr', 'shape'):
            pars.append(getattr(f.root, '%s_%s' % (name, par)).read())
    m = csr_matrix(tuple(pars[:3]), shape=pars[3])
    return m

In [47]:
store_sparse_mat(mat,"table")

store.h5 (File) ''
Last modif.: 'Fri Mar 24 15:13:44 2017'
Object Tree: 
/ (RootGroup) ''
/table_data (CArray(776397,)) ''
/table_indices (CArray(776397,)) ''
/table_indptr (CArray(10037,)) ''
/table_shape (CArray(2,)) ''

store.h5 (File) ''
Last modif.: 'Fri Mar 24 16:20:15 2017'
Object Tree: 
/ (RootGroup) ''
/table_data (CArray(776397,)) ''
/table_indices (CArray(776397,)) ''
/table_indptr (CArray(10037,)) ''
/table_shape (CArray(2,)) ''

store.h5 (File) ''
Last modif.: 'Fri Mar 24 16:20:15 2017'
Object Tree: 
/ (RootGroup) ''
/table_data (CArray(776397,)) ''
/table_indices (CArray(776397,)) ''
/table_indptr (CArray(10037,)) ''
/table_shape (CArray(2,)) ''

store.h5 (File) ''
Last modif.: 'Fri Mar 24 16:20:15 2017'
Object Tree: 
/ (RootGroup) ''
/table_data (CArray(776397,)) ''
/table_indices (CArray(776397,)) ''
/table_indptr (CArray(10037,)) ''
/table_shape (CArray(2,)) ''



In [37]:
matcheck=load_sparse_mat("table")

In [38]:
filecheck=matcheck.toarray()

In [35]:
sparasecheck=sparse_df_to_array(sparsefile)

In [36]:
store_sparse_mat(sparasecheck,"table")

In [42]:
type(filecheck)

numpy.ndarray

In [41]:
type(file)

pandas.core.frame.DataFrame

In [43]:
f=pd.HDFStore("store.h5")

In [45]:
f

<class 'pandas.io.pytables.HDFStore'>
File path: store.h5
Empty

In [ ]:
f